In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(patchwork)

In [ ]:
extra_theme <- 
theme(legend.position = "none") + 
  theme(axis.title.x = element_blank(),axis.title.y = element_blank()) + 
  theme(axis.text.y = element_text(size = 12), plot.title = element_text(size = 13)) 

# Prepare data for analysis

- Prepared categorical markers

In [ ]:
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

In [ ]:
go_results <- 
fread(paste0(SHARE_DIR, "2b_go.csv")) %>% 
 fi(group != "type") %>% 
 mu(or = ifelse(or == "Inf", exp(5), or), 
    p_bh = p.adjust(fisher_pval, method = "BH"),  
    p_by = p.adjust(fisher_pval, method = "BY")) %>% 
 fi(type != "combination")

- Barplot of Significance

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 10)

In [ ]:
threshold <- .05

- Organize the cohort names

In [ ]:
cohorts_map <- fread(paste0(SHARE_DIR, "cohort_names_map.csv"))

In [ ]:
base <- 
go_results %>% 
 ij(cohorts_map %>% se(cohortGo, cohortGoo, cohortGooo, pan), by = "cohortGo") %>% 
 mu(cohortGoo = factor(cohortGoo, levels = cohorts_map$cohortGoo[order(cohorts_map$ct)]))

In [ ]:
base$cohortGooo <-
factor(base$cohortGooo, 
       levels = base %>% gb(cohortGooo) %>% su(max_sig = max(-log10(fisher_pval))) %>% ar(desc(max_sig)) %>% pu(cohortGooo))

# 2 - Barplots for different cohorts

In [ ]:
plter <- function(d_base, title = "X", text_size = 4){
 dd <-   
 d_base %>% 
  gb(cohortGoo) %>% 
  su( p2 = mean(fisher_pval < threshold), p1 = mean(p_bh < threshold)) %>% 
  mu( p2 = p2 - p1) %>% 
  ga(gp, val, -cohortGoo)   

 dd_tot <- dd %>% gb(cohortGoo) %>% su(tot = sum(val))
    
 dd %>%
  ggplot(aes(y = cohortGoo, x = val, fill = gp)) + geom_bar(stat = "identity", color = "black") +   
  geom_vline(xintercept = threshold, color = "black", alpha = .3)+ 
  go_theme + extra_theme + labs(title = title) + scale_x_continuous(labels = scales::percent, expand = expansion(c(0, .3))) +
  geom_text(data = dd_tot, aes(y = cohortGoo, x = tot, label = paste0(round(signif(100*tot,2),1), "%")), inherit.aes = FALSE, hjust = -.2, size = 4) 
}

In [ ]:
p1 <- plter(base %>% fi(!pan, group == "treatment"), "Cohort Specific: Drug Treatment")
p2 <- plter(base %>% fi(pan, group == "treatment"), "Pan-Cancer: Drug Treatment") 
p3 <- plter(base %>% fi(!pan, group == "mechanism"), "Cohort Specific: Drug Mechanism") 
#p3 <- plter(base %>% fi(!pan, group == "type"), "Cohort Specific: Drug Type") #+ labs(title = "Cohort Specific: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) 
p4 <- plter(base %>% fi(pan, group == "mechanism"), "Pan-Cancer: Drug Mechanism") 
#p6 <- plter(base %>% fi(pan, group == "type"), "Pan-Cancer: Drug Type")  #+ labs(title = "Pan-Cancer: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) + scale_x_continuous(expand = expansion(c(0, 0.2))) 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)

In [ ]:
treatment <- (p1 / p2 ) + plot_layout(heights = c(3,5)) 
mechanism <- (p3 / p4) + plot_layout(heights = c(10,9)) 
oo = (treatment | mechanism ) + plot_layout(widths = c(1,1)) 

In [ ]:
oo 

In [ ]:
ggsave( paste0(SHARE_DIR, "2_volcano_summaries_sig_cohorts.png"), plot = oo, width = 14, height = 8)